# data.ca.gov

The Employment Development Department of the Staet of California offers a [CKAN](https://docs.ckan.org/en/latest/user-guide.html#what-is-ckan) data [API](https://data.ca.gov/api/1/util/snippet/api_info.html?resource_id=5642307f-30c2-4ddb-b811-507b338e0b4d) that can be queried using SQL (*likely SQLite*).

In [ ]:
# Dependencies:

!pip install fuzzywuzzy
!pip install urllib3

In [2]:
import pandas as pd
import requests

In [ ]:
def clean_sql(sql: str) -> str:
    """ 
    Helper function to clean SQL queries for requests to the CA Open Data Portal
    """
    return " ".join(sql.replace("\n", " ").split())

In [5]:
# Source: https://data.ca.gov/organization/california-employment-development-department
ca_labor_market_data = {
    "occupation": "274e273c-d18c-4d84-b8df-49b4d13c14ce",
    "industry": "5642307f-30c2-4ddb-b811-507b338e0b4d",
}

sql_url = "https://data.ca.gov/api/3/action/datastore_search_sql"

---

## Gathering the Data Element Dictionaries

The function below gathers the data element dictionaries from the CA Open Data Portal. These define the columns of the data sets.

In [6]:
def get_data_dictionary(data_type: str) -> pd.DataFrame:
    """
    Get the data dictionary for the specified data type from the CA Open Data
    Portal

    :param data_type: Either "occupation" or "industry"
    :return: A DataFrame containing the data dictionary
    """
    base_url = "https://data.ca.gov/dataset/"
    if data_type == "occupation":
        base_url += "long-term-occupational-employment-projections/resource/"
    elif data_type == "industry":
        base_url += "long-term-industry-employment-projections/resource/"
    else:
        raise ValueError(
            "Invalid data type. Must be either 'occupation' or 'industry'."
        )
    resource_id = ca_labor_market_data[data_type]
    url = f"{base_url}{resource_id}"
    return pd.read_html(url)[0] # [1] gives data recency information

In [7]:
get_data_dictionary("occupation")

,Column,Type,Label,Description
0,Area Type,text,area_type,"The type of geographic area (e.g., State, Coun..."
1,Area Name,text,area_name,The official name of the geographic area.
2,Period,text,period,The 4-digit calendar years of the base and pro...
3,SOC Level,numeric,soc_level,The level of detail of the occupational code.
4,Standard Occupational Classification (SOC),text,standard_occupational_classification_soc,The classification of workers into occupationa...
5,Occupational Title,text,occupational_title,The official occupation name.
6,Base Year Employment Estimate,numeric,base_year_employment_estimate,The number of jobs in the base year.
7,Projected Year Employment Estimate,numeric,projected_year_employment_estimate,The number of jobs in the projected year.
8,Numeric Change,numeric,numeric_change,Difference in employment between base and proj...
9,Percentage Change,numeric,percentage_change,Numeric change divided by base year employment.


In [8]:
get_data_dictionary("industry")

,Column,Type,Label,Description
0,Area Type,text,area_type,"The type of geographic area (e.g., State, Coun..."
1,Area Name,text,area_name,The official name of the geographic area.
2,Period,text,period,The 4-digit calendar years of the base and pro...
3,Series Code,text,series_code,The code identifying the specific series.
4,Industry Title,text,industry_title,The official industry name.
5,Base Year Employment Estimate,numeric,base_year_employment_estimate,The number of jobs in the base year.
6,Projected Year Employment Estimate,numeric,projected_year_employment_estimate,The number of jobs in the projected year.
7,Numeric Change,numeric,numeric_change,Difference in employment between base and proj...
8,Percentage Change,numeric,percentage_change,Numeric change divided by base year employment.


---

## Querying the Database

In [63]:
def labor_market_sql_to_dataframe(sql: str) -> pd.DataFrame:
    """
    Executes a SQL query against the CA Open Data Portal and converts the 
    result set to a DataFrame

    :param sql: The SQL query to execute
    :return: A DataFrame containing the results of the query
    """
    try:
        response = requests.get(sql_url, params={"sql": clean_sql(sql)})
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(f"Error raised with SQL:\n{sql}\nError message: {e}")
        return pd.DataFrame()

    data = response.json()
    return pd.DataFrame(data.get("result", {}).get("records", {}))

In [66]:
sql = f""" 
    SELECT
        "Occupational Title",
        ROUND(AVG("Numeric Change"), 2) AS "Average Projected Employment Change"
    FROM
        "{ca_labor_market_data['occupation']}"
    WHERE
        "SOC Level" = 2
        AND "Area Name" <> 'California'
    GROUP BY
        "Occupational Title"
    ORDER BY
        "Average Projected Employment Change" DESC
"""

In [67]:
df = labor_market_sql_to_dataframe(sql)
df.head()

,Occupational Title,Average Projected Employment Change
0,Food Preparation and Serving Related Occupations,14808.75
1,Transportation and Material Moving Occupations,9894.06
2,Healthcare Support Occupations,8740.63
3,Management Occupations,7585.00
4,Business and Financial Operations Occupations,5667.81


In [64]:
sql = f""" 
    SELECT DISTINCT
        "Occupational Title"
    FROM
        "{ca_labor_market_data['occupation']}"
    WHERE
        /* Test of comment */
        "SOC Level" = 4
"""

In [65]:
df = labor_market_sql_to_dataframe(sql)
df.head()

,Occupational Title
0,"Passenger Vehicle Drivers, Except Bus Drivers,..."
1,Food Preparation Workers
2,Rail Car Repairers
3,"Farmers, Ranchers, and Other Agricultural Mana..."
4,"Woodworking Machine Setters, Operators, and Te..."


In [78]:
sql = f""" 
    SELECT DISTINCT
        *
    FROM
        "{ca_labor_market_data['occupation']}"
    WHERE
        "SOC Level" = 3
"""

In [79]:
df = labor_market_sql_to_dataframe(sql)
df.sample(10)

,_id,_full_text,Area Type,Area Name,Period,SOC Level,Standard Occupational Classification (SOC),Occupational Title,Base Year Employment Estimate,Projected Year Employment Estimate,Numeric Change,Percentage Change,Exits,Transfers,Total Job Openings,Median Hourly Wage,Median Annual Wage,Entry Level Education,Work Experience,Job Training
222,1631,'-2032':4 '-9000':7 '0':20 '0.00':19 '11.5':15...,State,California,2022-2032,3,33-9000,Other Protective Service Workers,250100,278800,28700,11.5,208140,218190,455030,0.00,0,N/A,N/A,N/A
2045,11969,'-2030':20 '-9000':23 '0':38 '0.00':37 '10080'...,Metropolitan Area,San Francisco-Redwood City-South San Francisco...,2020-2030,3,39-9000,Other Personal Care and Service Workers,10460,13230,2770,26.5,7200,10080,20050,0.00,0,N/A,N/A,N/A
2877,16469,'-2030':10 '-5000':13 '0':29 '0.00':28 '1240':...,Metropolitan Area,Visalia-Porterville MSA (Tulare County),2020-2030,3,43-5000,"Material Recording, Scheduling, Dispatching, a...",2140,2350,210,9.8,820,1240,2270,0.00,0,N/A,N/A,N/A
2506,14499,'-2030':13 '-9000':16 '0':30 '0.00':29 '1330':...,Metropolitan Area,Santa Maria-Santa Barbara MSA (Santa Barbara C...,2020-2030,3,41-9000,Other Sales and Related Workers,1330,1550,220,16.5,860,890,1970,0.00,0,N/A,N/A,N/A
891,5247,'-1000':11 '-2030':8 '0':27 '0.00':26 '11.8':2...,Metropolitan Area,Madera MSA (Madera County),2020-2030,3,53-1000,Supervisors of Transportation and Material Mov...,170,190,20,11.8,70,130,220,0.00,0,N/A,N/A,N/A
2076,12149,'-2030':20 '-8000':23 '0':36 '0.00':35 '1030':...,Metropolitan Area,San Francisco-Redwood City-South San Francisco...,2020-2030,3,51-8000,Plant and System Operators,1030,1070,40,3.9,410,720,1170,0.00,0,N/A,N/A,N/A
1365,7628,'-1000':17 '-2030':14 '0':28 '0.00':27 '15970'...,Metropolitan Area,Oakland-Hayward-Berkeley MD (Alameda and Contr...,2020-2030,3,25-1000,Postsecondary Teachers,16000,16920,920,5.8,7360,7690,15970,0.00,0,N/A,N/A,N/A
2212,13034,'-10':29 '-2.4':30 '-2000':21 '-2030':18 '0':3...,Metropolitan Area,San Luis Obispo-Paso Robles-Arroyo Grande MSA ...,2020-2030,3,33-2000,Fire Fighting and Prevention Workers,420,410,-10,-2.4,100,210,300,0.00,0,N/A,N/A,N/A
2425,14058,'-2030':12 '-4000':15 '0':29 '0.00':28 '10':23...,Metropolitan Area,Santa Cruz-Watsonville MSA (Santa Cruz County),2020-2030,3,45-4000,"Forest, Conservation, and Logging Workers",70,80,10,14.3,30,90,130,0.00,0,N/A,N/A,N/A
2380,13844,'-2030':12 '-3000':15 '0':28 '0.00':27 '1000':...,Metropolitan Area,Santa Cruz-Watsonville MSA (Santa Cruz County),2020-2030,3,25-3000,Other Teachers and Instructors,1700,2040,340,20.0,1110,1000,2450,0.00,0,N/A,N/A,N/A


---

## Integration with CCC API

### CA Data

In [68]:
sql = f"""
    SELECT DISTINCT
        "Area Name"
    FROM 
        "{ca_labor_market_data['occupation']}"
"""

In [70]:
areas = labor_market_sql_to_dataframe(sql)
areas.head()

,Area Name
0,Redding MSA (Shasta County)
1,Oakland-Hayward-Berkeley MD (Alameda and Contr...
2,Fresno MSA (Fresno County)
3,San Jose-Sunnyvale-Santa Clara MSA (San Benito...
4,Riverside-San Bernardino-Ontario MSA (Riversid...


In [71]:
# Split the "Area Name" column at the parentheses
areas[['Metropolitan Statistical Areas', 'Counties']] = (
    areas['Area Name'].str.extract(r'^(.*) \((.*)\)$')
)

areas.head()

,Area Name,Metropolitan Statistical Areas,Counties
0,Redding MSA (Shasta County),Redding MSA,Shasta County
1,Oakland-Hayward-Berkeley MD (Alameda and Contr...,Oakland-Hayward-Berkeley MD,Alameda and Contra Costa Counties
2,Fresno MSA (Fresno County),Fresno MSA,Fresno County
3,San Jose-Sunnyvale-Santa Clara MSA (San Benito...,San Jose-Sunnyvale-Santa Clara MSA,San Benito and Santa Clara Counties
4,Riverside-San Bernardino-Ontario MSA (Riversid...,Riverside-San Bernardino-Ontario MSA,Riverside and San Bernardino Counties


### CCC Data

In [72]:
# Requests to https://api.cccco.edu/ always throw SSL warnings. 
# Suppress them here.

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [45]:
def get_colleges(search_param: str | None = None) -> pd.DataFrame:
    """
    Sends a `GET` request to the CCCCO API to retrieve a list of colleges.

    :param search_param: The parameters for the search query. Either the college
        MIS ID or the college name. If `None`, all colleges are returned.
    :return: A pandas DataFrame containing the list of colleges.
    """

    cccco_api_url = "https://api.cccco.edu/"

    if not search_param:
        cccco_api_url += "colleges"
    elif search_param.isnumeric():
        cccco_api_url += f"colleges/{search_param}"
    else:
        cccco_api_url += f"colleges/search/{search_param}"

    response = requests.get(cccco_api_url, verify=False)
    response.raise_for_status()
    return pd.DataFrame(response.json()).drop(columns=["CollegeContacts"]) 

In [46]:
colleges = get_colleges()

We'll need to (fuzzy) join the labor market data with the CCC data on the County fields.

In [47]:
from fuzzywuzzy import process


def get_best_match(county: str, choices: list[str]) -> str | None:
    """ 
    Uses fuzzy matching to find the best match for a given county in a list of 
    choices.

    :param county: The county to match.
    :param choices: The list of choices to match against.
    :return: The best match for the county in the list of choices.
    """
    best_match = process.extractOne(county, choices)
    return best_match[0] if best_match else None

c:\Users\codyc\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [50]:
area_counties = areas["Counties"].dropna().unique()

colleges["Matched Counties"] = colleges["County"].apply(
    lambda x: get_best_match(x, area_counties)
)

merged_df = pd.merge(
    colleges, areas, left_on="Matched Counties", right_on="Counties", how="left"
)

merged_df.head()

,CollegeID,CollegeName,DistrictID,StreetAddress,City,County,Zip,ZipPlus4,MailingAddress,MailingCity,...,Phone,WebsiteURL,Latitude,Longitude,LogoURL,District,Matched Counties,Area Name,Metropolitan Statistical Areas,Counties
0,021,Cuyamaca College,020,900 Rancho San Diego Parkway,El Cajon,San Diego,92019,4304,900 Rancho San Diego Parkway,El Cajon,...,619.660.4000,www.cuyamaca.edu,32.744890,-116.935229,CuyamacaCollegeLogo.jpg,None,San Diego County,San Diego-Carlsbad MSA (San Diego County),San Diego-Carlsbad MSA,San Diego County
1,022,Grossmont College,020,8800 Grossmont College Drive,El Cajon,San Diego,92020,1799,8800 Grossmont College Drive,El Cajon,...,619.644.7000,www.grossmont.edu,32.817897,-117.005640,GrossmontCollegelogo.jpg,None,San Diego County,San Diego-Carlsbad MSA (San Diego County),San Diego-Carlsbad MSA,San Diego County
2,031,Imperial Valley College,030,380 East Aten Road,Imperial,Imperial,92251,9787,380 East Aten Road,Imperial,...,760.352.8320,www.imperial.edu,32.825859,-115.502999,ImperialValleyCollegeLogocopy.jpg,None,Imperial County,El Centro MSA (Imperial County),El Centro MSA,Imperial County
3,051,MiraCosta College,050,1 Barnard Drive,Oceanside,San Diego,92056,3899,1 Barnard Drive,Oceanside,...,760.757.2121,www.miracosta.edu,33.188864,-117.301064,Mira_Costa_College_Logo_4c.png,None,San Diego County,San Diego-Carlsbad MSA (San Diego County),San Diego-Carlsbad MSA,San Diego County
4,061,Palomar College,060,1140 West Mission Road,San Marcos,San Diego,92069,1487,1140 West Mission Road,San Marcos,...,760.744.1150,www.palomar.edu,33.147015,-117.183980,PalomarCollegeLogo.jpg,None,San Diego County,San Diego-Carlsbad MSA (San Diego County),San Diego-Carlsbad MSA,San Diego County


In [ ]:
merged_df[
    [
        "CollegeID",
        "CollegeName",
        "County",
        "Matched Counties",
        "Area Name",
        "Metropolitan Statistical Areas",
    ]
].sample(10) # Merge was successful

,CollegeID,CollegeName,County,Matched Counties,Area Name,Metropolitan Statistical Areas
65,592,Modesto Junior College,Stanislaus,Stanislaus County,Modesto MSA (Stanislaus County),Modesto MSA
63,582,Lemoore College,Kings,Kings County,Hanford-Corcoran MSA (Kings County),Hanford-Corcoran MSA
78,741,Los Angeles City College,Los Angeles,Los Angeles County,Los Angeles-Long Beach-Glendale MD (Los Angele...,Los Angeles-Long Beach-Glendale MD
55,531,Merced College,Merced,Merced County,Merced MSA (Merced County),Merced MSA
32,343,Laney College,Alameda,Alameda and Contra Costa Counties,Oakland-Hayward-Berkeley MD (Alameda and Contr...,Oakland-Hayward-Berkeley MD
99,873,Santiago Canyon College,Orange County,Orange County,Anaheim-Santa Ana-Irvine MD (Orange County),Anaheim-Santa Ana-Irvine MD
113,982,San Bernardino Valley College,San Bernardino,Riverside and San Bernardino Counties,Riverside-San Bernardino-Ontario MSA (Riversid...,Riverside-San Bernardino-Ontario MSA
90,821,Citrus College,Los Angeles,Los Angeles County,Los Angeles-Long Beach-Glendale MD (Los Angele...,Los Angeles-Long Beach-Glendale MD
52,521,Bakersfield College,Kern County,Kern County,Bakersfield MSA (Kern County),Bakersfield MSA
74,691,Taft College,Kern County,Kern County,Bakersfield MSA (Kern County),Bakersfield MSA


---